In [ ]:
import random

import matplotlib.pyplot as plt
import numpy as np
import torch

import utilities_latent as ul
import utilities_plot as up
from utilities_base import VAE, load_data

In [ ]:
# 学習済みの重みとデータを読み込む

data, data_id_dict, child_id_dict, word_dict, category_dict = load_data(
    ["data", "data_id_dict", "child_id_dict", "word_dict", "category_dict"]
)
word_count = len(word_dict)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE().to(device)
model.eval()
model.load_state_dict(torch.load("tmp/best_model.pth"))
# model.load_state_dict(torch.load("tmp/model_state_dict.pth"))

In [ ]:
# カテゴリの表示
for key, val in category_dict.items():
    print(key, len(val))

In [ ]:
# 学習済みモデルで実データを潜在空間に射影

figs = {}
figs["tmp"] = plt.subplots()
fig, ax = figs["tmp"]
up.plot_x(model, data, ax)
ax.set_xlabel("z1")
ax.set_ylabel("z2")
fig.savefig("images/latent_space.png")

In [ ]:
# 年齢の実データと語彙の実データを潜在空間上にプロット

figs["age"] = plt.subplots()
figs["vocabulary"] = plt.subplots()

data_ids = list(data_id_dict.keys())
up.plot_x_with_age(model, data_ids, *figs["age"])
up.plot_x_with_vocabulary(model, data_ids, *figs["vocabulary"], ["all"])
# up.plot_x_with_vocabulary(
#     model,
#     data_ids,
#     *figs["vocabulary"],
#     [上のセルの結果を参考にしてここに語彙のカテゴリを入れる(["all"]だと全語彙)]
# )

In [ ]:
# 潜在空間の語彙の発達具合をcategoryごとに可視化して保存

z1_start, z1_end = -6, 7
z2_start, z2_end = -3, 3
spacing = 0.1

z_meshgrid = up.make_lattice_points(z1_start, z1_end, z2_start, z2_end, spacing)
for i, category in enumerate(category_dict.keys()):
    figs[category] = plt.subplots()
    up.plot_vocabulary(model, z_meshgrid, *figs[category], [category])
    up.plot_x(model, data, figs[category][1], "black")
    figs[category][0].savefig(f"images/vocabulary/{i}_{category}_vocabulary.png")
    plt.close()

In [ ]:
# 潜在空間の語彙の発達具合を全categoryまとめて可視化して保存

figs["all"] = plt.subplots()
up.plot_vocabulary(model, z_meshgrid, *figs["all"], ["all"])
up.plot_x(model, data, figs["all"][1], "black")
figs["all"][0].savefig("images/vocabulary/all_vocabulary.png")
plt.close()

In [ ]:
# 縦断データ(同じ子供の年齢が違うデータ)の可視化

figs["arrow"] = plt.subplots()
up.plot_x(model, data, figs["arrow"][1])
data_ids = []

# 何個以上の縦断データを選ぶか
n = 2
for i, v in child_id_dict.items():
    if len(v) >= n:
        data_ids.append([j[0] for j in v])
# print(len(data_ids))

# n個以上の縦断データの中からランダムにm個選んでプロット
m = 20
datas = random.sample(data_ids, m)
child_id = data_id_dict[datas[0][0]][0]
print(child_id_dict[child_id])
for i in datas:
    up.plot_arrow(model, i, figs["arrow"][1])
# figs["arrow"][0].savefig("images/arrow.png")

In [ ]:
# 観察しやすそうな実データ

# child_id: 3461, 3499, 2860
tmp_data_id = 3499
tmp_child_id = data_id_dict[tmp_data_id][0]
# print(tmp_child_id)
# print(child_id_dict[tmp_child_id])
data_ids = [i[0] for i in child_id_dict[tmp_child_id]]
print(data_ids)

In [ ]:
tmp_data = up.child_id_to_data(tmp_child_id)
tmp_z = up.x_to_z(model, tmp_data)
point_O = tmp_z[1, :]
point_A = tmp_z[2, :]
point_P = tmp_z[3, :]

# 直線OPに対してAと対称な点Bを求める
point_B = ul.get_symmetric_points(point_O, point_P, point_A)

In [ ]:
figs["tmp"] = plt.subplots()
fig, ax = figs["tmp"]
up.plot_x(model, data, ax)
up.plot_arrow(model, data_ids, ax)
points = {"O": point_O, "A": point_A, "P": point_P, "B": point_B}
for i in points:
    ax.scatter(points[i][0], points[i][1], color="tab:orange")
    ax.annotate(i, points[i], textcoords="offset points", xytext=(0, 5), ha="center")